# OCEAN Data NFTs
Quickly create OCEAN Data NFTs by querying blockchain data from different providers.

Supported Blockchain Data Providers:
* [Dune](https://dune.com/)
* [Flipside](https://flipsidecrypto.xyz/)

In [ ]:
import json
import pandas as pd
import os
from dotenv import load_dotenv
import json
from flipside import Flipside
from ocean_lib.ocean.ocean import Ocean

# Load variables from .env file
load_dotenv()

# Access environment variables
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")

## Flipside

URL: https://api.flipsidecrypto.com/api/v2/queries/f8e505dc-172b-44e5-99a6-8c35fe0695a5/data/latest

In [ ]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(flipside_key, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns)
    return results_df

In [ ]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
order by 1 asc
"""

results_df = query_flipside(sql)
results_df

## Dune

URL: https://api.dune.com/api/v1/query/2296642/results?api_key=XXX

## OCEAN Protocol

https://docs.oceanprotocol.com/developers/ocean.py/install

https://stackoverflow.com/questions/76707475/issue-importing-pyyaml-cltk